In [1]:
def init_environment(forzar_drive=False):
  # 1) Montar Google Drive
  from google.colab import drive
  drive.mount('/content/drive', force_remount=forzar_drive)

  # 2) Rutas base
  from pathlib import Path
  BASE_DIR     = Path('/content/drive/MyDrive/structure')
  MODELS_DIR   = BASE_DIR / 'models'
  NOTEBOOKS_DIR= BASE_DIR / 'notebooks'
  CONFIGS_DIR  = BASE_DIR / 'configs'
  DATA_DIR     = BASE_DIR / 'datasets/processed'
  LOGS_DIR     = BASE_DIR / 'logs'

  for d in (MODELS_DIR, NOTEBOOKS_DIR, LOGS_DIR):
      d.mkdir(parents=True, exist_ok=True)

  # 3) Path y sys.path
  import sys
  sys.path.append(str(MODELS_DIR))
  sys.path.append(str(BASE_DIR))

In [ ]:
# =============================================================================
# 🚀 Notebook principal de experimentación con TensorFlow/Keras
# =============================================================================

# 1) Preparar el entorno
#    - Monta Google Drive
#    - Ajusta sys.path para encontrar tus módulos en models/ y utils/
init_environment(forzar_drive=True)

# 2) Definir experimento
#    El nombre debe coincidir con un archivo YAML en configs/experiments/
exp_name = "<EXP NAME>"

# 3) Cargar configuración y datos
from utils.experiment.functions import load_experiment
cfg, NNClass, params, dataset, train_data, val_data, val_data_idx = load_experiment(exp_name)


# 4) Instanciar el modelo
#    BaseTFModel se encarga de build_model(), compile() y callbacks
model = NNClass(cfg, **params)


# 5) Mostrar resumen de la configuración
from utils.misc.functions import print_exp_configuration
print(f"\n✔️ Experimento «{cfg['experiment']['name']}» cargado con éxito.\n")
print_exp_configuration(cfg)

# 6) Mostrar arquitectura del modelo
print("\n📋 Arquitectura del modelo:")
model.model.summary()

# 7) Entrenar (o retomar) hasta la época final
history = model.fit(train_data, val_data)

# 8) Evaluación final en validación
loss, acc = model.model.evaluate(val_data, verbose=2)
print(f"\n🔍 Resultados finales — Loss: {loss:.4f}, Accuracy: {acc:.4f}")

# 9) Análisis
from utils.analysis.analysis import ExperimentAnalyzer

analyzer = ExperimentAnalyzer(
    model=model.model,
    history=history,
    val_data=val_data_idx,                    # tf.data.Dataset con índices
    cfg=cfg,
    effects=dataset.get_effects("val")        # Structured array
)

# gráficos habituales
analyzer.plot_training_curves()
analyzer.confusion_matrix(normalize="true")

# nuevos diagnósticos
analyzer.effect_diagnostics("num_taps")
analyzer.effect_diagnostics("snr_db")
analyzer.effect_diagnostics("phase_offset")
analyzer.effect_diagnostics("roll_off")

Mounted at /content/drive
⬇️  Descargando «carlosandres069/snr8-12-taps2-phase0-45» …
Dataset URL: https://www.kaggle.com/datasets/carlosandres069/snr8-12-taps2-phase0-45


100%|██████████| 748M/748M [00:10<00:00, 78.0MB/s]



✅ Usando archivo HDF5: datasets/kaggle/snr8_12_taps2_phase0_45.hdf5
✔️ Modelo Keras inicializado

✔️ Experimento «exp_resnet1d_4block» cargado con éxito.


=== EXPERIMENTO ===
╒═══════════════╤═══════════════════════════════════════════════════════════════════════╕
│ Campo         │ Valor                                                                 │
╞═══════════════╪═══════════════════════════════════════════════════════════════════════╡
│ Nombre        │ exp_resnet1d_4block                                                   │
├───────────────┼───────────────────────────────────────────────────────────────────────┤
│ Descripción   │ ResNet1D profundo de 4 bloques para clasificación de modulaciones AMC │
├───────────────┼───────────────────────────────────────────────────────────────────────┤
│ Módulo modelo │ resnet1d_4block                                                       │
├───────────────┼───────────────────────────────────────────────────────────────────────┤
│ Clase model

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 4096, 2)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 4096, 64)  │        960 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 4096, 64)  │     20,544 │ conv1d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_2 (Conv1D)   │ (None, 4096, 64)  │     20,544 │ conv1d_1[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 4096, 64)  │          0 │ conv1d[0][0],     │
│                     │                   │            │ conv1d_2[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 4096, 64)  │          0 │ add[0][0]         │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_3 (Conv1D)   │ (None, 4096, 64)  │     20,544 │ activation[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_4 (Conv1D)   │ (None, 4096, 64)  │     20,544 │ conv1d_3[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 4096, 64)  │          0 │ activation[0][0], │
│                     │                   │            │ conv1d_4[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 4096, 64)  │          0 │ add_1[0][0]       │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_5 (Conv1D)   │ (None, 4096, 64)  │     20,544 │ activation_1[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_6 (Conv1D)   │ (None, 4096, 64)  │     20,544 │ conv1d_5[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 4096, 64)  │          0 │ activation_1[0][… │
│                     │                   │            │ conv1d_6[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 4096, 64)  │          0 │ add_2[0][0]       │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_7 (Conv1D)   │ (None, 4096, 64)  │     20,544 │ activation_2[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_8 (Conv1D)   │ (None, 4096, 64)  │     20,544 │ conv1d_7[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 4096, 64)  │          0 │ activation_2[0][… │
│                     │                   │            │ conv1d_8[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 4096, 64)  │          0 │ add_3[0][0]       │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 64)        │          0 │ activation_3[0][… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 7)         │        455 │ global_average_p

...

 Trainable params: 165,767 (647.53 KB)

 Non-trainable params: 0 (0.00 B)

🔄 Cargando checkpoint previo: epoch_06.keras


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 42 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


✔️ Pesos cargados desde /content/drive/MyDrive/structure/outputs/resnet1d_4block/checkpoints/epoch_06.keras
Epoch 7/100
287/287 ━━━━━━━━━━━━━━━━━━━━ 0s 310ms/step - accuracy: 0.4800 - loss: 1.1517
Epoch 7: val_accuracy improved from -inf to 0.47079, saving model to /content/drive/MyDrive/structure/outputs/resnet1d_4block/checkpoints/epoch_07.keras
287/287 ━━━━━━━━━━━━━━━━━━━━ 102s 327ms/step - accuracy: 0.4800 - loss: 1.1517 - val_accuracy: 0.4708 - val_loss: 1.1610
Epoch 8/100
287/287 ━━━━━━━━━━━━━━━━━━━━ 0s 306ms/step - accuracy: 0.4862 - loss: 1.1350
Epoch 8: val_accuracy improved from 0.47079 to 0.49416, saving model to /content/drive/MyDrive/structure/outputs/resnet1d_4block/checkpoints/epoch_08.keras
287/287 ━━━━━━━━━━━━━━━━━━━━ 90s 314ms/step - accuracy: 0.4862 - loss: 1.1350 - val_accuracy: 0.4942 - val_loss: 1.1233
Epoch 9/100
